# EBA3500 Lecture 7. Interaction and power

We'll use the `titanic` data set once again. 

In [ ]:
import seaborn as sns
titanic = sns.load_dataset("titanic")
titanic.head()

## Interactions

Interactions are regression terms that involve products of two or more covariates, e.g., 
$$y = \alpha + \beta_1 x_1 + \beta_{12} x_1x_2.$$ 
The terminology is most freqently used for categorical variables. Let's explore what it means.

## The Cartesian product

We'll have to deal with another fundamental set-theoretic construction, called the *Cartesian product*. 

**Note:** If something is *Cartesian*, it is named after the French reneissance philosopher René Descartes.

We will use the symbol "$\in$", which reads "in". Writing "$a\in A$" means the same as "$a \textrm{ in } A$". This symbol may look unfamiliar but is a staple of mathematical notation.

#### Definition
> Let $A$ and $B$ be two sets. The *Cartesian product*, denoted by $A\times B$, is the unique set containing the tuples $(a,b)$ for $a\in A$ and $b \in B$. 

We also call $A \times B$ the *product set*, or the product of $A$ and $B$. 

#### Example

Recall the sex and class categories from the Titanic data set.

In [ ]:
set(titanic.sex)

In [ ]:
set(titanic["class"])

We will construct the product set of these two sets. To to this, we will use the `product` function from `itertools`. This function often comes in handy in programming in general.

In [ ]:
from itertools import product
sex_class = set(product(set(titanic.sex), set(titanic["class"])))
sex_class

It is not the case that $A\times B = B\times A$!

In [ ]:
from itertools import product
class_sex = set(product(set(titanic["class"]), set(titanic["sex"])))
class_sex

In [ ]:
class_sex == sex_class

We can also take the product of more than two sets, say $A\times B \times C$. Such products are defined in the obvious way, i.e., $(a,b,c)\in A\times B\times C$ if and only if $a\in A$, $b \in B$, and $c \in C$.

In [ ]:
set(titanic.survived)

In [ ]:
set(product(set(titanic["class"]), set(titanic["sex"]), set(titanic["survived"])))

#### Exercise
Suppose that $A, B, C$ have elements $n,m,k$ each. What is the size of $A\times B\times C$?

### Titanic and interaction

Recall from the last lecture that both sex and class affects the probability of survival. 

In [ ]:
import seaborn as sns
sns.catplot(x="sex", hue="survived", col="class", data=titanic, kind="count")

The basic model involves only *linear effects*
$$
y = \beta_0 + \beta_{\textrm{male}}1[\textrm{sex} = \textrm{male}] + \beta_{\textrm{2nd}}1[\textrm{class} = \textrm{2nd}]+ \beta_{\textrm{3rd}}1[\textrm{class} = \textrm{3rd}].
$$



Let's fit it and see what we get.

In [ ]:
import statsmodels.formula.api as smf
fit = smf.ols("survived ~ sex + Q('class')", data = titanic).fit()
fit.summary()

Now we check how well the estimates match the data. First we'll take a look at the predicted values of the model.

In [ ]:
import pandas as pd
records = sorted(list(product(set(titanic.sex), set(titanic["class"]))))
records

We need to make this into a data frame, as follows.

In [ ]:
frame = pd.DataFrame.from_records(records, columns = ["sex", "class"])
frame

We want to predict the probability of survival at the values in `frame`.
So to do this, we will use the `predict` method.

In [ ]:
fit.predict(frame)

This is just a list of floats with no names. Let's merge it with the `frame` to make stuff clearer

In [ ]:
frame["predicted"] = fit.predict(frame)
frame

Let's compare these predictions to the means using the `groupby` method.

In [ ]:
means = titanic.groupby(["sex", "class"]).mean()["survived"]
means

This table is simple enough. The mean on the column to the right is the mean of the subgroup for the corresponding sex and class on the right.

Let's compare these means to the predicted values of our model.

In [ ]:
import numpy as np
frame["mean"] = means.array
frame

Why aren't the values equal? First, notice that there are $6$ means, but only $4$ parameters in the regression output. 

In [ ]:
fit.params




Second, the model doens't take *interactions* into account, only linear effects. Interactions occur when the effect of one variable depends on the value of another variable.


### A "two-way" model

Let $A$ and $B$ be sets with $I$ and $J$ elements each. Consider the following model
$$y=\sum_{i=1}^{I}\sum_{j=1}^{J}\beta_{ij}1[X_{1}=a_{j}]1[X_{2}=b_{k}].$$
That is the same model as, using $X = (X_1, X_2)$,
$$y=\sum_{i=1}^{I}\sum_{j=1}^{J}\beta_{ij}1[X = (a_{j}, b_{k})],$$
where $(a_i,b_j) \in A\times B$. Thus it may be viewed as an ordinary categorical variable model.

### Titanic example

Let $A = \{\textrm{female}, \textrm{male}\}$ and $B = \{\textrm{1st},\textrm{2nd}, \textrm{3rd}\}$. That is, $A$ are the sex categories and $B$ are the class categories. Remember their product set, which equals


In [ ]:
set(product(set(titanic["sex"]), set(titanic["class"])))

Writing down the model explicitly doesn't help, as it's cumbersome to look at. But using $$y=\sum_{i=1}^{I}\sum_{j=1}^{J}\beta_{ij}1[X = (a_{j}, b_{k})],$$
where $(a_i,b_j) \in A\times B$, we can write a table of the corresponding coefficients.

|         |        | Class |     |     |
| ---     |------- | ----- | --- |-----| 
|         |        | 1st   | 2nd | 3rd |
| **Sex** |        |       |     |     |
|         | Female | $\beta_{11}$ | $\beta_{12}$ | $\beta_{13}$ | 
|         | Male   | $\beta_{21}$ | $\beta_{22}$ | $\beta_{23}$ |

When you want to predict the probability of survival for a male passenger traveling 3rd class, you find the second row (male) and third column (3rd class), and use the $\beta_{23}$ coefficient. 

Now let's fit the model! To do this, we will define a new column in the data frame `titanic` containing the product set of `Sex` and `Class`.

In [ ]:
titanic["sex_class"] = list(zip(titanic["sex"], titanic["class"]))
titanic["sex_class"]

Now we're ready to run the model.

In [ ]:
fit2 = smf.ols("survived ~ sex_class - 1", data = titanic).fit()
fit2.summary()

Let's see how these new coefficients match the means we just calculated.

In [ ]:
frame["new_predicted"] = fit2.params.array
frame

That's perfect!

As it happens, `statsmodels` allows us to run interaction models such as these easily. We just need to use the `*` command:

In [ ]:
import statsmodels.formula.api as smf
fit = smf.ols("survived ~ sex * Q('class')", data = titanic).fit()
fit.summary()

Yet again, we have to think about baselines. Since there is no `sex[T.Female]` in the table, the baseline sex is female. Since there is no `class[T.1st]` in the table, the baseline class is 1st class.

To calculate the predicted value of, say, a male with class `3rd`, you would have to sum the coefficients of `sex[T.Male]`, `class[T.3rd]` *and* `sex[T.Male]:class[T.3rd]`. 

In [ ]:
fit.params

In [ ]:
coef = fit.params
coef["Intercept"] + coef["sex[T.male]"] + coef["Q('class')[T.Third]"]+ coef["sex[T.male]:Q('class')[T.Third]"]

In [ ]:
frame["mean"][5]

It's clearly harder to interpret the coefficients of a the `statsmodels` object with formula `"survived ~ sex * class"` than that with formula `"survived ~ sex_class - 1"`. However, we may use the formulation `"survived ~ sex * class"` to more easily ask meaningful questions.




Most importantly, do we need the interaction terms or not? In other words, do we need the whole model `"survived ~ sex * Q('class')"`, or would it suffice with `"survived ~ sex + Q('class')"`?

## Testing for interactions

Testing for interactions is similar to the testing we did in the last lecture. Consider a model with $2$ categorical variables $x_1$ and $x_2$ with $I$ and $J$ categories each. The model with linear effects is 
$$
y = \beta_0 + \sum_{i = 2}^{I} {\beta^1_{i}1[x_1 = i]} + \sum_{j = 2}^{J} {\beta^2_{j} 1[x_2 = j]} + \epsilon
$$

Here we start the sums at $i=2$ and $j=2$ since the $x_1 = 1$ and $x_2 = 2$ are absorbed into the baseline.

This is the general form of the titanic model with two categories we've already looked at:

In [ ]:
fit = smf.ols("survived ~ sex + Q('class')", data = titanic).fit()
fit.summary()

This model doesn't include interactions, as there are no product terms $1[x_1 = i]1[x_2 = j]$ in the model. Let's add them to the model and see what it looks like.

The interaction model $$y=\sum_{i=1}^{I}\sum_{j=1}^{J}\beta_{ij}1[X_{1}=a_{j}]1[X_{2}=b_{k}]$$ can be written as en extension of the linear effects model $$y = \beta_0 + \sum_{i = 2}^{I} {\beta^1_{i}1[x_1 = i]} + \sum_{j = 2}^{J} {\beta^2_{j} 1[x_2 = j]} + \sum_{i = 2}^{I}\sum_{j = 2}^{J} {\beta^{1\times2}_{ij} 1[x_1 = j]1[x_2=j]} + \epsilon$$

The double sum starts at $i,j=2$ due to absorption into the baseline.

Formulating the interaction model in this way allows us to formulate a test of whether the interactions matter. Our null hypothesis is that the all the regression coefficients belonging to the interaction part of the model are $0$. Formally, we could write
$$H_0: \beta^{1\times2}_{ij} = 0\quad \textrm{for all } i,j\geq2.$$
Our alternative hypothesis is that the interaction terms matter *in some way*. In other words, at least one coefficients $\beta^{1\times2}_{ij}$ is not equal to $0$.
$$H_a: \textrm{There is a pair } (i,j) \textrm{ where } \beta^{1\times2}_{ij} \neq 0.$$

Notice that we only care about the interaction coefficients $\beta_{ij}^{1\times2}$; the other regression coefficients aren't featured in the hypothesis statements at all.

This is similar to the setup in the previous lecture in the $F$ test, except that we care about a *subset* of $\beta$

### Example

We'll continue work with the interaction model for `titanic`.

First, recall the linear effects model:
$$
y = \beta_0 + \beta_{\textrm{male}}1[\textrm{sex} = \textrm{male}] + \beta_{\textrm{2nd}}1[\textrm{class} = \textrm{2nd}]+ \beta_{\textrm{3rd}}1[\textrm{class} = \textrm{3rd}].
$$

In the `statsmodels` regression model below, we add the interaction terms corresponding to $\beta_{\textrm{male}\times\textrm{2nd}}$, $\beta_{\textrm{male}\times\textrm{3rd}}$, making the interaction model

$$
y = \beta_0 + \beta_{\textrm{male}}1[\textrm{sex} = \textrm{male}] + \beta_{\textrm{2nd}}1[\textrm{class} = \textrm{2nd}]+ \beta_{\textrm{3rd}}1[\textrm{class} = \textrm{3rd}] + 
\beta_{\textrm{male}\times\textrm{2nd}}1[(\textrm{sex}, \textrm{class}) = (\textrm{male}, \textrm{2nd})] + 
\beta_{\textrm{male}\times\textrm{3rd}}1[(\textrm{sex}, \textrm{class}) = (\textrm{male}, \textrm{3nd})].
$$

In [ ]:
import statsmodels.formula.api as smf
fit = smf.ols("survived ~ sex * Q('class')", data = titanic).fit()
fit.summary()

Now we'll make what is called an ANOVA table.

In [ ]:
import statsmodels.api as sm
sm.stats.anova_lm(fit, typ=3) 

Let's interpret this table. It contains three *p*-values, one for `sex`, one for `class`, and one for `sex:class`, which is the interaction term. You only need to think about the $PR(>F)$ column, which contains the *p*-values.

1. **Sex.** Here $H_0$ is that all the regression coefficients for the category `Sex` are $0$ (that is, is $\beta_\textrm{male} = 0$) when we do not include any of the other coefficients in the model. I.e., it ask if sex matters when we do not include `Class` and interactions.
2. **Class.** Now $H_0$ is that all the regression coefficients for the category `Class` are $0$ when we do include the coefficients for `Sex` in the model.
3. **Sex:Class.** $H_0$ is that all the interaction coefficients, i.e., the coefficients for the category `Sex:Class` are $0$ when the coefficients for `Sex` and `Class` are included in the model.

In conclusion, we find that the *p*-value for our null hypothesis $H_0$, that all of the interaction regression coefficients are $0$, is `6.545202e-16`, or $6.54\cdot10^{-16}$.

## Marketing example

Inference problems with multiple categorical variables and their interactions are often called ANOVA, or "analysis of variance". Very prominently used in research, but also in business. [Here is an example in marketing](https://imsmwu.github.io/MRDA2017/_book/analysis-of-variance.html). Notice that most people work with various sums of squares when dealing with ANOVA. We *don't* do that in this course.    

> Assume that you are a marketing manager at an online fashion store, and you wish to analyze the effect of online promotion on sales. You conduct an experiment and select a sample of 30 comparable products to be included in the experiment. Then you randomly assign the products to one of three groups: (1) = high promotion level, (2) = medium promotion level, (3) = low promotion level, and record the sales over one day. This means that you have 10 products assigned to each treatment.

We download the data and take a look at it.

In [ ]:
import pandas as pd
sales = pd.read_table("https://raw.githubusercontent.com/IMSMWU/Teaching/master/MRDA2017/online_store_promo.dat")
sales.head()

In [ ]:
sales.info()

Let's visualize this.

In [ ]:
sns.catplot(y = "Sales", col = "Newsletter", x = "Promotion", kind = "bar", data = sales)

It seems like both newsletter and promotion has and effect on the amount of sales.

In [ ]:
model = smf.ols("Sales ~ Newsletter + Promotion", data = sales).fit()
sm.stats.anova_lm(model, typ=3) 

But let's check the interactions too!

In [ ]:
model = smf.ols("Sales ~ Newsletter * Promotion", data = sales).fit()
sm.stats.anova_lm(model, typ=3) 

Hence we have evidence that both inclusion in a newsletter and degree of promotion matter by themselves - and there is an interaction!

## $n$-way ANOVA
We have looked at two-way ANOVA: Here we have two categorical covariates and look at their interactions. We can also look at interactions between more than two variables. But this is seldom done, as it is hard to estimate them well.


In [ ]:
titanic_model = smf.ols("survived ~ sex * Q('class') * embarked", data = titanic).fit()
titanic_model.summary()

In [ ]:
titanic.shape

The total number of estimated coefficients is $2\cdot 3 \cdot 3$, as there are two sexes, three classes, and three ports. This is a lot to estimate, and the precision of the estimates becomes worse as a consequence.

In [ ]:
sm.stats.anova_lm(titanic_model, typ=3) 

It seems like `embarked` barely matters at all. So we try:

In [ ]:
sm.stats.anova_lm(smf.ols("survived ~ Q('class') * sex", data = titanic).fit(), typ=3) 

Usually a two-way design is enough. If you need to do more, be careful and read up on three-way ANOVA first.

## Power analysis

It's often convenient to frame experimental questions as hypothesis tests. The most common testing scenario is the two-sample problem. Here you have two means $\mu_0$ and $\mu_1$ and iid observations $X_1,X_2,\ldots,X_{n_1}$ from the first group and $Y_1,Y_2,\ldots Y_{n_2}$ from the second group. You wish to test:
$$\begin{align}
H_{0} & : \mu_{0}=\mu_{1},\\
H_{1} & : \mu_{0}\neq\mu_{1}.
\end{align}$$

This is usually done using the [*t*-test](https://en.wikipedia.org/wiki/t-test), which is an exact test if both $X_i$ and $Y_i$ are normal with the same standard deviation. In other words, $X_i\sim N(\mu_1,\sigma)$ and $Y_i\sim N(\mu_2, \sigma)$.

We have already looked at such tests with regression, but with the same sample sizes. Now we look at the case of unequal sample sizes using `scipy`.

**Power**- When we conduct an experiment we would like to have enough data to make it plausible that the null hypothesis gets rejected. The power of a test is defined as the probability of rejecting the null. It depends both on the unknown parameters and the sample size. We usually aim for a power of $80\%$; we do it by guessing the value of the unknown parameters and then simulate or calculate how many observations we need. 

### Simulation

We make a function that simulates from the normal model described above.

In [ ]:
from numpy import random
from scipy.stats import ttest_ind
rng = random.default_rng(seed = 313)

def simulate(n1, n2, mu1, mu2):
    """Simulate two normal vectors with means mu1, mu2 and standard deviation 1."""
    x = rng.normal(size = n1) + mu1
    y = rng.normal(size = n2) + mu2
    return (x, y)

Using unpacking we find $x$ and $y$.

In [ ]:
x, y = simulate(10, 20, mu1 = 1, mu2 = 1.2)
print((x, y))

And we can use the `ttest_ind` function from `scipy` to calculate the $t$-test. 

In [ ]:
ttest_ind(x, y)

We combine these two simulations to simulate *p*-values directly.

In [ ]:
def check_power(n1, n2, mu1, mu2):
    """Simulate p-value using simulate."""
    x, y = simulate(n1, n2, mu1, mu2)
    return ttest_ind(x, y).pvalue

In [ ]:
check_power(10, 20, 1, 1.2)

Now we can make a histogram of the resulting simultions when $\mu_1 = 1$ and $\mu_2 = 1.2$.

In [ ]:
pvalues = np.array([check_power(10, 20, 1, 1.2) for _ in range(n_reps)])
sns.histplot(pvalues, stat = "probability")
np.mean(pvalues <= 0.5)

We do not have high enough sample size to get a big probability of rejection! Let's double the sample size and see what happens.

In [ ]:
pvalues = np.array([check_power(20, 40, 1, 1.2) for _ in range(n_reps)])
sns.histplot(pvalues, stat = "probability")
np.mean(pvalues <= 0.5)

Still not a big chance of rejecting $H_0$.

### Using `statsmodels`.
We an use `statsmodels` to calculate power too, but only for the more simple situation with one mean and one sequence of variables. It uses the notion of effect size, which equals $(\mu_2 - \mu_1)/\sigma$. Since $\sigma = 1$ in our case, our effect size is $0.2$.


In [ ]:
from statsmodels.stats.power import TTestIndPower
power = TTestIndPower()
power.solve_power(effect_size = 0.2, power = 0.8, nobs1 = None, alpha=0.5)

This suggest that we need about $n=100$ observations in both groups to get a power of $80\%$.

In [ ]:
pvalues = np.array([check_power(80, 100, 1, 1.2) for _ in range(n_reps)])
sns.histplot(pvalues, stat = "probability")
np.mean(pvalues <= 0.5)